In [1]:
import pandas as pd

In [2]:
!pip install kaggle

In [3]:
# Create the .kaggle directory
!mkdir -p ~/.kaggle

# Move kaggle.json to the .kaggle directory
!mv kaggle.json ~/.kaggle/

# Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [4]:
!kaggle datasets download -d dineshpiyasamara/sentiment-analysis-dataset

Dataset URL: https://www.kaggle.com/datasets/dineshpiyasamara/sentiment-analysis-dataset
License(s): unknown
  0% 0.00/460k [00:00<?, ?B/s]
100% 460k/460k [00:00<00:00, 81.6MB/s]


In [5]:
!mkdir data
!unzip sentiment-analysis-dataset.zip -d data
!ls data

Archive:  sentiment-analysis-dataset.zip
  inflating: data/sentiment_analysis.csv  
sentiment_analysis.csv


In [6]:
data = pd.read_csv('data/sentiment_analysis.csv')
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [7]:
tweet_test = data['tweet'][0]
tweet_test

'#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone'

In [51]:
from pprint import PrettyPrinter
import requests
pp = PrettyPrinter()
pp.pprint(requests.get("http://sentiment-546041470502.asia-southeast2.run.app/v1/models/sentiment").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1733577019'}]}


In [54]:
import tensorflow as tf
import base64
import json


def prepare_json(tweet):
    feature_spec = {
        "tweet": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(tweet, "utf-8")])),
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
        "inputs" : result
    })

In [58]:
response = requests.post("https://sentiment-546041470502.asia-southeast2.run.app/v1/models/sentiment:predict", data=prepare_json(tweet_test))
prediction = response.json().get("predictions")

print(response)

<Response [400]>


In [43]:
print(prepare_json(tweet_test))

{"signature_name": "serving_default", "instances": {"examples": "CpMBCpABCgV0d2VldBKGAQqDAQqAASNmaW5nZXJwcmludCAjUHJlZ25hbmN5IFRlc3QgaHR0cHM6Ly9nb28uZ2wvaDFNZlFWICNhbmRyb2lkICNhcHBzICNiZWF1dGlmdWwgI2N1dGUgI2hlYWx0aCAjaWdlcnMgI2lwaG9uZW9ubHkgI2lwaG9uZXNpYSAjaXBob25l"}}


In [11]:
print(prepare_json(tweet_test))

{"signature_name": "serving_default", "instances": [{"examples": {"b64": "CpMBCpABCgV0d2VldBKGAQqDAQqAASNmaW5nZXJwcmludCAjUHJlZ25hbmN5IFRlc3QgaHR0cHM6Ly9nb28uZ2wvaDFNZlFWICNhbmRyb2lkICNhcHBzICNiZWF1dGlmdWwgI2N1dGUgI2hlYWx0aCAjaWdlcnMgI2lwaG9uZW9ubHkgI2lwaG9uZXNpYSAjaXBob25l"}}]}
